In [2]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import stats
import sklearn
from sklearn.preprocessing import KBinsDiscretizer

In [14]:
# Adding KBM value to df
users = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/original_data/users_activated_v7.csv', low_memory = False)
march = pd.read_csv('C:/Users/sgulbin/Work/_Ad-hoc_analysis/users_for_ftr_score.csv', delimiter = ';')
# users = users[['user_id', 'sex', 'birthday', 'LicenseSetDate']]
# users.columns = ['user_id', 'sex', 'birthday', 'license_set_date']
users = users[['user_id', 'user_ext','login', 'sex', 'birthday', 'license_set_date']]
kbm = pd.read_csv('C:/Users/sgulbin/Work/Data/KBM_data/KBM_data_5_27_2020.csv')
# feb = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/FTR_score/users_feb/users_feb.csv', delimiter = ';')
# march = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/FTR_score/users_march/users_march.csv', delimiter = ';')
df = pd.merge(pd.merge(march,users,left_on = 'user_ext', right_on = 'user_ext', how = 'left'), kbm, on = 'user_ext', how = 'left')
df['user_id'] = df['user_id_y']
# df = pd.merge(users, kbm, on = 'user_id', how = 'left')
df['age'] = np.clip((pd.to_datetime('today')-pd.to_datetime(df.birthday, errors = 'coerce')).astype('timedelta64[Y]'),19.0,80.0)
df['exp'] = np.clip((pd.to_datetime('today')-pd.to_datetime(df.license_set_date, errors = 'coerce')).astype('timedelta64[Y]'),1.0,62.0)
df['sex'] = df['sex'].fillna('Unknown')
df

,user_id_x,user_ext,user_id_y,login,sex,birthday,license_set_date,user_id,KBM,age,exp
0,20169745,522800,20169745,79031237789,М,1985-07-31,2003-01-01,20169745,0.50,34.0,17.0
1,832750152,27178645,832750152,79818940650,М,1997-06-15,2015-01-01,832750152,0.90,23.0,5.0
2,20929374,1304743,20929374,79852361903,Ж,1989-10-16,2011-01-01,20929374,0.85,30.0,9.0
3,111750116,23273833,111750116,79161009056,М,1984-04-11,2002-01-01,111750116,0.80,36.0,18.0
4,182001420,23880849,182001420,79262335906,М,1996-03-20,2017-01-01,182001420,0.95,24.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
776845,19981339,708417,19981339,79263046777,М,1976-06-03,1999-01-01,19981339,0.90,44.0,21.0
776846,1006500036,30133644,1006500036,79534215252,М,1985-11-04,2005-01-01,1006500036,NaN,34.0,15.0
776847,20267085,815239,20267085,79605244442,Unknown,1990-09-05,2011-01-01,20267085,NaN,29.0,9.0
776848,859501694,27488890,859501694,79169200277,Ж,1987-08-06,2005-01-01,859501694,1.00,32.0,15.0


In [9]:
# Generating age-exp coefficients
K_age_exp_max = 1.2
K_age_exp_min = 0.725
K_intercept = -4.88430556234637
K_f_age = 1.26397937674965
K_f_exp = 0.0762464696907463
f_age_exp_min = K_f_age*np.log(18)+K_f_exp*np.log(0.001)+K_intercept
f_age_exp_max = K_f_age*np.log(50)+K_f_exp*np.log(20)+K_intercept
K = K_age_exp_max-((K_f_age*np.log(18)+K_f_exp*np.log(0.001)+K_intercept)-f_age_exp_min)*(K_age_exp_max-K_age_exp_min)/(f_age_exp_max-f_age_exp_min)
ages = []
exps = []
koefs = []
for age in range(18,101):
    for exp in range(0,83):
        K = np.clip(K_age_exp_max-((K_f_age*np.log(age)+K_f_exp*np.log(exp)+K_intercept)-f_age_exp_min)*(K_age_exp_max-K_age_exp_min)/(f_age_exp_max-f_age_exp_min),K_age_exp_min,K_age_exp_max)
        ages.append(age)
        exps.append(exp)
        koefs.append(K)
age_exp_df = pd.DataFrame(data = ({'age':ages, 'exp':exps, 'K_age_exp': koefs}))
test = age_exp_df.loc[(age_exp_df.age == 30)&(age_exp_df.exp == 0)]
test
# print(age_exp_df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


,age,exp,K_age_exp
996,30,0,1.2


In [17]:
# Adding coefficients to df
df_K_sex = pd.DataFrame(data = ({'sex': ['М', 'Ж', 'Unknown'], 'K_sex': [1.0,1.10,1.0]}))
df_K_KBM = pd.DataFrame(data = ({'KBM': [0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0,1.4,1.55,2.3,2.45,np.nan],
                                 'K_KBM': [0.9,0.9,0.9,0.9,0.9,0.9,0.95,0.95,1.0,1.0,1.03,1.1,1.1,1.1,1.1,1.03]}))
df = pd.merge(df, df_K_sex, how = 'left', left_on = 'sex', right_on = 'sex')
df = pd.merge(df, df_K_KBM, how = 'left', left_on = 'KBM', right_on = 'KBM')
df = pd.merge(df, age_exp_df, how = 'left', left_on = ['age','exp'], right_on = ['age','exp'])
df['K_age_exp'] = np.where(df['age'].isnull(), 1.0, np.where(df['exp'].isnull(),1.0,np.where((df['age']-df['exp'])<16,1.0,df['K_age_exp'])))
df['K_age_exp'] = df['K_age_exp'].fillna(1.0)
df['K_KBM'] = df['K_KBM'].fillna(1.0)

df['K_KBM_model_score'] = np.clip(df.K_sex*df.K_KBM*df.K_age_exp,0.7,1.299999)
df

,user_id_x,user_ext,user_id_y,login,sex,birthday,license_set_date,user_id,KBM,age,exp,K_sex,K_KBM,K_age_exp,K_KBM_model_score
0,20169745,522800,20169745,79031237789,М,1985-07-31,2003-01-01,20169745,0.50,34.0,17.0,1.0,0.90,0.888587,0.799729
1,832750152,27178645,832750152,79818940650,М,1997-06-15,2015-01-01,832750152,0.90,23.0,5.0,1.0,1.00,1.080809,1.080809
2,20929374,1304743,20929374,79852361903,Ж,1989-10-16,2011-01-01,20929374,0.85,30.0,9.0,1.1,0.95,0.956232,0.999262
3,111750116,23273833,111750116,79161009056,М,1984-04-11,2002-01-01,111750116,0.80,36.0,18.0,1.0,0.95,0.863517,0.820341
4,182001420,23880849,182001420,79262335906,М,1996-03-20,2017-01-01,182001420,0.95,24.0,3.0,1.0,1.00,1.075950,1.075950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776845,19981339,708417,19981339,79263046777,М,1976-06-03,1999-01-01,19981339,0.90,44.0,21.0,1.0,1.00,0.776662,0.776662
776846,1006500036,30133644,1006500036,79534215252,М,1985-11-04,2005-01-01,1006500036,NaN,34.0,15.0,1.0,1.03,0.891710,0.918462
776847,20267085,815239,20267085,79605244442,Unknown,1990-09-05,2011-01-01,20267085,NaN,29.0,9.0,1.0,1.03,0.970255,0.999363
776848,859501694,27488890,859501694,79169200277,Ж,1987-08-06,2005-01-01,859501694,1.00,32.0,15.0,1.1,1.03,0.916788,1.038721


In [18]:
# Mapping coefficient with score
mapa = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/FTR_score_mapping_v5.1.csv')
mapa.columns = ['ftr_score', 'from', 'to']
mapa.index=pd.IntervalIndex.from_arrays(left=mapa['from'],right=mapa['to'],closed= 'left')
df = df.assign(ftr_score=mapa.loc[df.K_KBM_model_score].ftr_score.values)
df = df[['user_id','user_ext','ftr_score']]
# df.columns = ['user_id', 'user_ext', 'ftr_score']
df.to_csv('C:/Users/sgulbin/Work/_Ad-hoc_analysis/users_for_ftr_score_output.csv', index = False)
df

,user_id,user_ext,ftr_score
0,20169745,522800,0.88
1,832750152,27178645,0.22
2,20929374,1304743,0.41
3,111750116,23273833,0.84
4,182001420,23880849,0.23
...,...,...,...
776845,19981339,708417,0.92
776846,1006500036,30133644,0.62
776847,20267085,815239,0.41
776848,859501694,27488890,0.31


In [61]:
# Adding KBM value to dfr
usrs = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/original_data/users_activated_v6.csv', low_memory = False)
usrs = users[['user_id', 'login', 'sex', 'birthday', 'license_set_date']]
dfr = pd.merge(usrs, kbm, on = 'user_id', how = 'left')
dfr['age'] = np.clip((pd.to_datetime('today')-pd.to_datetime(dfr.birthday, errors = 'coerce')).astype('timedelta64[Y]'),19.0,80.0)
dfr['exp'] = np.clip((pd.to_datetime('today')-pd.to_datetime(dfr.license_set_date, errors = 'coerce')).astype('timedelta64[Y]'),1.0,62.0)
dfr['sex'] = dfr['sex'].fillna('Unknown')

# Adding coefficients to dfr
dfr_K_sex = pd.DataFrame(data = ({'sex': ['М', 'Ж', 'Unknown'], 'K_sex': [1.0,1.10,1.0]}))
dfr_K_KBM = pd.DataFrame(data = ({'KBM': [0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0,1.4,1.55,2.3,2.45,np.nan],
                                 'K_KBM': [0.9,0.9,0.9,0.9,0.9,0.9,0.95,0.95,1.0,1.0,1.03,1.1,1.1,1.1,1.1,1.03]}))
dfr = pd.merge(dfr, dfr_K_sex, how = 'left', left_on = 'sex', right_on = 'sex')
dfr = pd.merge(dfr, dfr_K_KBM, how = 'left', left_on = 'KBM', right_on = 'KBM')
dfr = pd.merge(dfr, age_exp_df, how = 'left', left_on = ['age','exp'], right_on = ['age','exp'])
dfr['K_age_exp'] = np.where(dfr['age'].isnull(), 1.0, np.where(dfr['exp'].isnull(),1.0,np.where((dfr['age']-dfr['exp'])<16,1.0,dfr['K_age_exp'])))
dfr['K_age_exp'] = dfr['K_age_exp'].fillna(1.0)
dfr['K_KBM'] = dfr['K_KBM'].fillna(1.0)
dfr['K_KBM_model_score'] = np.clip(dfr.K_sex*dfr.K_KBM*dfr.K_age_exp,0.7,1.3)

# Creating KBM_model_coefficient to KBM_model score mapping. Percentiles
est = KBinsDiscretizer(n_bins=103, encode='ordinal', strategy='quantile')
est.fit(dfr[['K_KBM_model_score']])
Xt = est.transform(dfr[['K_KBM_model_score']])
dfr['percentile'] = Xt
dfr



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


,user_id,login,sex,birthday,license_set_date,user_ext,KBM,age,exp,K_sex,K_KBM,K_age_exp,K_KBM_model_score,percentile
0,19500001,79039686761,М,1981-11-13,2006-01-01,29.0,1.00,38.0,14.0,1.0,1.03,0.847423,0.872845,27.0
1,19500014,79856864080,М,1989-01-26,NaN,94.0,0.80,31.0,NaN,1.0,0.95,1.000000,0.950000,47.0
2,19500017,79857600842,Ж,1984-05-18,2005-01-01,101.0,0.90,35.0,15.0,1.1,1.00,0.879720,0.967692,51.0
3,19500021,79853192128,М,1981-08-09,1998-01-01,128.0,0.85,38.0,22.0,1.0,0.95,0.836144,0.794337,11.0
4,19500023,79262821237,М,1991-02-05,NaN,133.0,0.50,29.0,NaN,1.0,0.90,1.000000,0.900000,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842617,1202000120,79850089239,Ж,1997-12-17,2016-01-01,NaN,NaN,22.0,4.0,1.1,1.03,1.104764,1.251698,98.0
1842618,1202000172,79616095924,М,1981-01-07,1997-01-01,NaN,NaN,39.0,23.0,1.0,1.03,0.824290,0.849019,22.0
1842619,1202000178,79219234478,М,1995-03-22,2017-01-01,NaN,NaN,25.0,3.0,1.0,1.03,1.059064,1.090836,80.0
1842620,1202000182,79851190609,М,2001-04-04,2019-01-01,NaN,NaN,19.0,1.0,1.0,1.03,1.200000,1.236000,97.0


In [12]:
# test = pd.merge(df,dfr, on = 'user_id', how = 'left')
# test = test[['User_ext', 'percentile']]
# test.columns = ['User_ext', 'ftr_score']
# test.ftr_score = test.ftr_score/100.0
# test.to_csv('C:/Users/sgulbin/Work/_Выгрузки_и_Расчеты/users_for_ftr_score_output.csv')
np.log(0.001)

-6.907755278982137